# Memory Information

In [1]:
import psutil
def get_size(bytes, suffix="B"):
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f"{bytes:.2f}{unit}{suffix}"
        bytes /= factor
print("="*40, "Memory Information", "="*40)
svmem = psutil.virtual_memory()
print(f"Total: {get_size(svmem.total)}") ; print(f"Available: {get_size(svmem.available)}")
print(f"Used: {get_size(svmem.used)}") ; print(f"Percentage: {svmem.percent}%")

======================================== Memory Information ========================================
Total: 25.51GB
Available: 24.59GB
Used: 591.92MB
Percentage: 3.6%


# GPU Information

In [2]:
! nvidia-smi

Fri Sep 25 19:39:28 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install -r requirements.txt

     |████████████████████████████████| 1.0MB 5.8MB/s 
     |████████████████████████████████| 61kB 7.3MB/s 
     |████████████████████████████████| 1.1MB 39.2MB/s 
     |████████████████████████████████| 3.0MB 47.5MB/s 
     |████████████████████████████████| 890kB 47.9MB/s 
     |████████████████████████████████| 102kB 9.3MB/s 
  Created wheel for pytreebank: filename=pytreebank-0.2.7-cp36-none-any.whl size=37070 sha256=8a3600335072dcc8939cf7f2252f05346f4a6e3f01a79d33a99fa1e73f6232df
  Stored in directory: /root/.cache/pip/wheels/e0/b6/91/e9edcdbf464f623628d5c3aa9de28888c726e270b9a29f2368
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=34427775dfff2fae2d04b7b03207ae950f44cf1cd5b02064dacc46e149bbc0fb
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7666 sha256=698f6cf4a5af5d7b14ae67d12f344a776c5bae69e

In [4]:
from transformers import GPT2Tokenizer, GPT2Model, GPT2Config
import torch
from dataset import SSTDataset
from torch.utils.data import DataLoader
from utils import transformer_params
from utils import evaluation_metrics, save_model, root_and_binary_title
from math import ceil
from loguru import logger
import numpy as np
import os
import time
from datetime import timedelta
from tqdm import tqdm

In [6]:
class GPT2ForSequenceClassification(torch.nn.Module):
  def __init__(self, tokenizer, num_labels):
    super(GPT2ForSequenceClassification, self).__init__()
    self.model = GPT2Model.from_pretrained('gpt2',
                                       config=GPT2Config.from_pretrained('gpt2'))
    self.tokenizer = tokenizer
    self.dropout = torch.nn.Dropout(p=0.1)
    self.fc_layer = torch.nn.Linear(in_features=768, out_features=768)
    self.tanh = torch.nn.GELU()
    self.out_layer = torch.nn.Linear(in_features=768, out_features=num_labels)
    self.criterion = torch.nn.CrossEntropyLoss()

  def forward(self, input_ids, attention_mask, labels):
    gpt_last_layer = self.model(input_ids, attention_mask=attention_mask)[0]
    #[batch_size, seq_len, embedding_size(channels)] = [*, *, 768]

    sep_mask = input_ids.eq(self.tokenizer.sep_token_id)
    if len(torch.unique(sep_mask.sum(1))) > 1:
      raise ValueError("All examples must have the same number of <e> tokens.")

    gpt_e_token = gpt_last_layer[sep_mask, :]
    #gpt_first_token = gpt_last_layer[:, 0] #experiment failed for first token
    #[batch_size, embedding_size(channels)] = [*, 768]

    fc_layer_out = self.fc_layer(gpt_e_token)
    fc_layer_out = self.tanh(fc_layer_out)
    #[batch_size, embedding_size(channels)] = [*, 768]

    fc_layer_out = self.dropout(fc_layer_out)
    logits = self.out_layer(fc_layer_out)
    #[batch_size, embedding_size(channels)] = [*, num_labels]
    
    loss = self.criterion(logits, labels)
                                   
    return logits, loss


In [16]:
class GPT2ForSequenceClassification(torch.nn.Module):
  def __init__(self, tokenizer, num_labels):
    super(GPT2ForSequenceClassification, self).__init__()
    self.model = GPT2Model.from_pretrained('gpt2',
                                       config=GPT2Config.from_pretrained('gpt2'))
    self.tokenizer = tokenizer
    self.dropout = torch.nn.Dropout(p=0.1)
    self.out_layer = torch.nn.Linear(in_features=768, out_features=num_labels)
    self.criterion = torch.nn.CrossEntropyLoss()

  def forward(self, input_ids, attention_mask, labels):
    gpt_last_layer = self.model(input_ids, attention_mask=attention_mask)[0]
    #[batch_size, seq_len, embedding_size(channels)] = [*, *, 768]

    sep_mask = input_ids.eq(self.tokenizer.sep_token_id)
    if len(torch.unique(sep_mask.sum(1))) > 1:
      raise ValueError("All examples must have the same number of <e> tokens.")

    gpt_e_token = gpt_last_layer[sep_mask, :]
    #gpt_first_token = gpt_last_layer[:, 0] #experiment failed for first token
    #[batch_size, embedding_size(channels)] = [*, 768]

    fc_layer_out = self.dropout(gpt_e_token)
    logits = self.out_layer(fc_layer_out)
    #[batch_size, embedding_size(channels)] = [*, num_labels]
    
    loss = self.criterion(logits, labels)
                                   
    return logits, loss


In [7]:
def load_transformer(name, binary):
  num_classes = 5
  if binary:
    num_classes = 2
  tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
  tokenizer.add_special_tokens({'pad_token': '[PAD]',
                              'cls_token': '<s>',
                              'sep_token': '<e>'})
  model = GPT2ForSequenceClassification(tokenizer, num_classes)
  model.model.resize_token_embeddings(len(tokenizer))

  return {'model': model,
          'tokenizer': tokenizer}

In [8]:
def add_special_tokens(text):
  text = list(text)
  for i in range(len(text)):
    text[i] = '<s> ' + text[i] + ' <e>'
  return tuple(text)

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [10]:
def train_step(model, inputs, labels, optimizer):
    optimizer.zero_grad()

    logits, loss = model(inputs['input_ids'], attention_mask=inputs['attention_mask'], labels=labels)

    loss.backward()
    optimizer.step()

    return logits, loss

In [11]:
def eval_step(model, inputs, labels):
    logits, loss = model(inputs['input_ids'], attention_mask=inputs['attention_mask'], labels=labels)

    return logits, loss

In [12]:
def train_epoch(model, tokenizer, train_dataset, optimizer, batch_size):
    train_loader = DataLoader(dataset=train_dataset,
                              batch_size=batch_size,
                              shuffle=True)

    correct_count = 0
    total_loss = 0

    model.train()
    with tqdm(total=ceil(len(train_dataset)/batch_size), desc='train', unit='batch') as pbar:
        for text, sentiment in train_loader:
            text = tokenizer(add_special_tokens(text), padding=True, return_tensors='pt').to(device)
            sentiment = sentiment.to(device)

            logits, loss = train_step(model, text, sentiment, optimizer)

            preds = torch.argmax(logits, axis=1)
            correct_count += (preds == sentiment).sum().item()
            total_loss += loss.item()
            pbar.update(1)

    return correct_count / len(train_dataset), total_loss / len(train_dataset)

In [13]:
def eval_epoch(model, tokenizer, eval_dataset, batch_size, split):
    eval_loader = DataLoader(dataset=eval_dataset,
                            batch_size=batch_size,
                            shuffle=True)

    correct_count = 0
    total_loss = 0
    y_pred = list()
    y_true = list()

    model.eval()
    with torch.no_grad():
        with tqdm(total=ceil(len(eval_dataset)/batch_size), desc=split, unit='batch') as pbar:
            for text, sentiment in eval_loader:
                text = tokenizer(add_special_tokens(text), padding=True, return_tensors='pt').to(device)
                sentiment = sentiment.to(device)

                logits, loss = eval_step(model, text, sentiment)

                preds = torch.argmax(logits, axis=1)
                y_pred += preds.cpu().numpy().tolist()
                y_true += sentiment.cpu().numpy().tolist()

                correct_count += (preds == sentiment).sum().item()
                total_loss += loss.item()
                pbar.update(1)

    metrics_score = evaluation_metrics(y_true, y_pred, split=split)
    return correct_count / len(eval_dataset), total_loss / len(eval_dataset), metrics_score

In [14]:
def train(name, root, binary, epochs=25, patience=3, save=False):

    #load model and tokenizer..
    try:
        transformer_container = load_transformer(name, binary)
    except ValueError:
        logger.error("Invalid transformer name!")
        os._exit(0)
    model = transformer_container['model']
    model = model.to(device)
    tokenizer = transformer_container['tokenizer']

    #load batch_size and learning rate..
    params_container = transformer_params(name)
    batch_size = params_container['batch_size']
    learning_rate = params_container['learning_rate']

    #load train, dev and test datasets..
    train_dataset = SSTDataset(root=root, binary=binary, split='train')
    dev_dataset = SSTDataset(root=root, binary=binary, split='dev')
    test_dataset = SSTDataset(root=root, binary=binary, split='test')

    #Intialize optimizer..
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    #Initialize training variables..
    best_acc = 0.0
    best_loss = np.inf
    stopping_step = 0
    best_model_name = None

    total_train_seconds = 0
    for epoch in range(epochs):

        start = time.time()
        train_acc, train_loss = train_epoch(model, tokenizer, train_dataset, optimizer, batch_size)
        end = time.time()
        total_train_seconds += (end - start)
        logger.info(f"epoch: {epoch+1}, transformer: {name}, train_loss: {train_loss:.4f}, train_acc: {train_acc*100:.2f}")

        dev_acc, dev_loss, _ = eval_epoch(model, tokenizer, dev_dataset, batch_size, 'dev')
        logger.info(f"epoch: {epoch+1}, transformer: {name}, dev_loss: {dev_loss:.4f}, dev_acc: {dev_acc*100:.2f}")

        test_acc, test_loss, test_evaluation_metrics = eval_epoch(model, tokenizer, test_dataset,
                                                                  batch_size, 'test')
        logger.info(f"epoch: {epoch+1}, transformer: {name}, test_loss: {test_loss:.4f}, test_acc: {test_acc*100:.2f}")
        logger.info(f"epoch: {epoch+1}, transformer: {name}, "
                    f"test_precision: {test_evaluation_metrics['test_precision']*100:.2f}, "
                    f"test_recall: {test_evaluation_metrics['test_recall']*100:.2f}, "
                    f"test_f1_score: {test_evaluation_metrics['test_f1_score']*100:.2f}, "
                    f"test_accuracy_score: {test_evaluation_metrics['test_accuracy']*100:.2f}")
        logger.info(f"epoch: {epoch+1}, transformer: {name}, test_confusion_matrix: \n"
                    f"{test_evaluation_metrics['test_confusion_matrix']}")

        logger.info(f"Total training time elapsed: {timedelta(seconds=total_train_seconds)}")
        logger.info(f"Mean time per train epoch: {timedelta(seconds=total_train_seconds/(epoch+1))}")

        #save best model and delete previous ones...
        if save:
            if test_acc > best_acc:
                best_acc = test_acc
                phrase_type, label = root_and_binary_title(root, binary)
                model_name = "{}_{}_{}_{}.pickle".format(name, phrase_type, label, epoch)
                save_model(model, model_name, best_model_name)


        # Implement early stopping here
        if test_loss < best_loss:
            best_loss = test_loss
            stopping_step = 0
        else:
            stopping_step += 1

        if stopping_step >= patience:
            logger.info("EarlyStopping!")
            os._exit(1)


In [15]:
#with one hidden layer
train('gpt2', True, False, 30, 300, False)

Some weights of GPT2Model were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2020-09-25 19:45:47.338 | INFO     | dataset:__init__:17 - Preparing dataset config root: True, binary: False, split: train!


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


2020-09-25 19:45:59.098 | INFO     | dataset:__init__:17 - Preparing dataset config root: True, binary: False, split: dev!
2020-09-25 19:46:05.048 | INFO     | dataset:__init__:17 - Preparing dataset config root: True, binary: False, split: test!
train: 100%|██████████| 267/267 [00:55<00:00,  4.85batch/s]
2020-09-25 19:47:00.787 | INFO     | __main__:train:39 - epoch: 1, transformer: gpt2, train_loss: 0.0513, train_acc: 26.32
dev: 100%|██████████| 35/35 [00:02<00:00, 15.19batch/s]
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
2020-09-25 19:47:03.124 | INFO     | __main__:train:42 - epoch: 1, transformer: gpt2, dev_loss: 0.0492, dev_acc: 27.97
test: 100%|██████████| 70/70 [00:04<00:00, 14.81batch/s]
2020-09-25 19:47:07.869 | INFO     | __m

KeyboardInterrupt: ignored

In [17]:
#no hidden layer
train('gpt2', True, False, 30, 300, False)

Some weights of GPT2Model were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2020-09-25 19:57:50.682 | INFO     | dataset:__init__:17 - Preparing dataset config root: True, binary: False, split: train!
2020-09-25 19:57:58.620 | INFO     | dataset:__init__:17 - Preparing dataset config root: True, binary: False, split: dev!
2020-09-25 19:58:04.651 | INFO     | dataset:__init__:17 - Preparing dataset config root: True, binary: False, split: test!
train: 100%|██████████| 267/267 [00:53<00:00,  4.95batch/s]
2020-09-25 19:58:59.261 | INFO     | __main__:train:39 - e

KeyboardInterrupt: ignored